In [1]:
%load_ext autoreload  
%autoreload 2 

In [2]:
from src.subset_classifier import *
import pandas as pd
from datasets import load_metric
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification
import uuid
import numpy as np
import torch
import wandb
from scipy.special import softmax

wandb.init(project="subset_selection_active_learning", entity="johnny-gary")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: garylai (johnny-gary). Use `wandb login --relogin` to force relogin


In [3]:
# DB_PATH = "./subset_selection/sst_results.db"
# df = get_df_from_db(DB_PATH)

In [4]:
# read df from pickle file
df = pd.read_pickle("./results/sst_results_df.pkl")

In [5]:
subset_unique_counts = get_subset_unique_counts(df); print(subset_unique_counts)

Counter({96: 3164, 93: 1815, 94: 1793, 95: 876, 92: 661, 97: 262})


In [6]:
optimal_subset_data_indices = get_optimal_subset_data_indices(df)

In [7]:
config = OptimalSubsetClassifierConfig(max_length=66, debug=False, model_name="albert-base-v2", batch_size=8, max_steps=40000)

In [8]:
train_ds, valid_ds, test_ds, debug_ds = create_train_valid_test_debug_ds(optimal_subset_data_indices, config)

print(len(train_ds), len(valid_ds), len(test_ds), len(debug_ds))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?ex/s]

INFO:src.subset_classifier:dataset      num positive examples    num negative examples
---------  -----------------------  -----------------------
train                           77                      723
valid                            9                       91
test                            10                       90


800 100 100 12


In [9]:
metric = load_metric("roc_auc")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    logits = softmax(logits, axis=1)
    predictions = logits[:, 1]
    return metric.compute(prediction_scores=predictions, references=labels)

In [10]:
config.debug

False

In [11]:
# config.debug = True; print(config)
training_args = TrainingArguments(
            output_dir="./subset_classifier_results_auc",
            max_steps=config.max_steps if not config.debug else 10000,
#             max_steps=31,
            evaluation_strategy="steps",
            report_to="wandb", 
            run_name=f"subset_classifier_result_{uuid.uuid4().hex}",
            logging_steps=300,
#             logging_steps=30,
            eval_steps=300,
#             eval_steps=30,
            learning_rate=1e-7,
            adam_epsilon=1e-6,
            warmup_ratio=0.1,
            weight_decay=0.01,
        )

In [12]:
import torch
print(training_args.run_name, training_args.max_steps, training_args.report_to, torch.cuda.is_available(), config)

subset_classifier_result_228e1353e225410fb6538a058f771ec3 40000 ['wandb'] True OptimalSubsetClassifierConfig(max_length=66, debug=False, model_name='albert-base-v2', batch_size=8, max_steps=40000)


In [13]:
model = AutoModelForSequenceClassification.from_pretrained(config.model_name, num_labels=2)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds if not config.debug else debug_ds,
    eval_dataset=valid_ds if not config.debug else debug_ds,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [14]:
# out = model(**{k: torch.unsqueeze(v, dim=0).to("cuda") for k, v in train_ds[0].items() if k != "labels"})

In [15]:
trainer.train()

C:\Users\t-johnnywei\Anaconda3\envs\torch\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 800
  Num Epochs = 400
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 40000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Roc Auc
300,0.711900,0.697491,0.416361
600,0.692000,0.662115,0.435897
900,0.645900,0.605960,0.446886
1200,0.582800,0.528251,0.472527
1500,0.498700,0.454382,0.496947
1800,0.433700,0.402984,0.463980
2100,0.386300,0.363028,0.471306
2400,0.347700,0.334162,0.490842
2700,0.322000,0.316687,0.495726
3000,0.304900,0.308375,0.520147


***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-500
Configuration saved in ./subset_classifier_results_auc\checkpoint-500\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-1000
Configuration saved in ./subset_classifier_results_auc\checkpoint-1000\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-1000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-1500
Configuration saved in ./subset_classifier_results_auc\checkpoint-1500\config.json
Model 

Model weights saved in ./subset_classifier_results_auc\checkpoint-11500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-12000
Configuration saved in ./subset_classifier_results_auc\checkpoint-12000\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-12000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-12500
Configuration saved in ./subset_classifier_results_auc\checkpoint-12500\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-12500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkp

  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-23000
Configuration saved in ./subset_classifier_results_auc\checkpoint-23000\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-23000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-23500
Configuration saved in ./subset_classifier_results_auc\checkpoint-23500\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-23500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-24000
Configuration saved in ./subset_classifier_results_auc\checkpoint-24000\config.json
Model weights saved in ./subset_classifier_resu

Model weights saved in ./subset_classifier_results_auc\checkpoint-34000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-34500
Configuration saved in ./subset_classifier_results_auc\checkpoint-34500\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-34500\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkpoint-35000
Configuration saved in ./subset_classifier_results_auc\checkpoint-35000\config.json
Model weights saved in ./subset_classifier_results_auc\checkpoint-35000\pytorch_model.bin
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
***** Running Evaluation *****
  Num examples = 100
  Batch size = 8
Saving model checkpoint to ./subset_classifier_results_auc\checkp

TrainOutput(global_step=40000, training_loss=0.10869532250761986, metrics={'train_runtime': 4607.4168, 'train_samples_per_second': 69.453, 'train_steps_per_second': 8.682, 'total_flos': 985796697600000.0, 'train_loss': 0.10869532250761986, 'epoch': 400.0})

In [16]:
1 + 1

2